# TPM034A Machine Learning for socio-technical systems 
## `Lab session 02: Artificial Neural Networks`

**Delft University of Technology**<br>
**Q2 2022**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are used in practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 

### `Workspace set-up`
**Option 1: Google Colab**<br>
Uncomment the following cells code lines if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/TPM34A/Q2_2022
#!pip install -r Q2_2022/requirements_colab.txt
#!mv "/content/Q2_2022/Lab_sessions/lab_session_02/data" /content/data

**Option 2: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment. This will install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

### `Application: Mode choice prediction` <br>
In this lab session, we will use ANN to predict mode-choice behaviour. In transportation, policy-makers often estimate the potential change in mode-choice behaviour resulting from a proposed policy, such as fuel levies, subsidization of public transport, or speed limits. Additionally, they use it for planning transport infrastructure. Based on travel demand forecast e.g. roads are widened or new train connections are built. Traditionally, mode choice predictions are based on classic econometric tools. Recently, the high prediction performance of ML methods appeals to researchers to use ML for predicting choice behaviour too. In this lab session, you will develop ANNs to predict mode behaviour and assess its performance.<br>


**Learning objectives**. After completing the following exercises you will be able to: <br>
1. `Prepare data` for training
2. Train `MultiLayerPerceptron` (MLP) - a particular type of artifical neural network - for a classification task<br>
3. `Tune` the `hyperparameters` to improve the model performance<br>
4. `Assess the performance` of competing models based on various performance measures, including confusion matrices, and Precision, Recall and Matthew's coefficient<br>

#### `Organisation`
This lab session comprises **`6`** parts:
1. Preparing (choice) data for training Artificial Neural Networks
2. Training a MultiLayerPerceptron (MLP) neural network
3. Using Early stopping to avoid overfitting
4. Using k-fold cross validation to evaluate generalisation performance
5. Tuning hyperparameters
6. Evaluating performance of trained models

and comprises **`7`** `exercises`.

In [ ]:
# Import required Python packages and modules
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Import selected functions and classes from Python packages
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay, log_loss, matthews_corrcoef, make_scorer, classification_report

# Setting
pd.set_option('display.max_columns', None)

#### 1. Preparing (choice) data for training Artificial Neural Networks<br>
To prepare the data set, we will:<br>
- 1.1 **Load** the data set<br>
- 1.2 **Inspect** and **Clean** the data set<br>
- 1.3 **Discover and visualise** the data <br>

##### 1.1. Load the choice data<br>

In [ ]:
# Load mode choice data into a pandas DataFrame
data_folder = Path('data')
df = pd.read_csv(data_folder/'swissmetro.dat', sep = '\t')

##### 1.2 Inspect and clean the data<br>
Before starting to analyse your data, make sure you understand what features are in your data.<br>
Therefore, it is `highly recommended` to look at the description of the data set. [Click here](data/CS_SwissmetroDescription.pdf) to open the pdf.

In [ ]:
# Inspect the range and distribution statistics 
df.describe()

# Inspect the data types in the df
# df.info()

In [ ]:
# Data cleaning

# Only keep data for purposes 'Commute" and "Business"
df.drop(df[(df.PURPOSE != 1) & (df.PURPOSE != 3)].index, inplace=True) 

# Drop rows with unknown choices (CHOICE == 0)
df.drop(df[df.CHOICE == 0].index, inplace=True) 

# In case of missing values, we replace them with 0
df.fillna(0, inplace = True) 

##### 1.3 Discovering and visualising the data<br>
Before starting to analyse your data with models, it is advisable to start with some **descriptive analyses**.<br>
Therefore, we first look at the distribution and correlations of key features in the data.

### <span style="color:skyblue">Exercise 1: Is the data set (im)balanced?</span> 
`A` Create a histogram showing how often TRAIN, SM and CAR are chosen. Do not forget to add labels to the columns<br>
`B` Interpret the the histogram. Is the data set imbalanced? <br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 4))
axes = sns.countplot(x = "CHOICE", data = df)
ylabels = ['Train', 'SM', 'Car']
axes.set_xticklabels(ylabels)

### <span style="color:skyblue">Exercise 2: Explore correlations between features</span> 
`A` Create a heatmap to identify what features particularly correlate with the **CHOICE**<br>
`B` Identify the features that strongly correlate (corr >0.9) (if any). Do they make sense? <br>
Hint: to do so you will need the description of the features [Click here](https://github.com/TPM34A/Q2_2022/blob/main/Lab_sessions/lab_session_02/data/CS_SwissmetroDescription.pdf) to open the pdf.

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

#### 2. Training a MultiLayerPerceptron (MLP)
To train an MLP (or any other sorts of artifical neural networks) we must take the following steps:<br>
- 2.1 **Scaling** the features<br>
- 2.2 **Splitting** the data in a training and test data set<br>
- 2.3 **Creating the model object**<br>
- 2.4 **Training** the model on the train data<br>
- 2.5 **Evaluating** the performance on the test data

##### 2.1 Scaling the features<br>
To efficiently train ANNs it strongly recommended to **scale** (a.k.a. normalise) the features. There are several ways to scale your data. A commonly used scaler of `sk-learn` is called 'StandardScaler'. This scaler normalises the variance and shift the location of the distribution to zero, see https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Create the list of features that we want to use in the model
features = ['PURPOSE', 'FIRST', 
            'TICKET', 'WHO', 'LUGGAGE', 'AGE', 
            'MALE', 'INCOME', 'GA', 'ORIGIN', 
            'DEST', 'TRAIN_TT', 'TRAIN_CO', 
            'TRAIN_HE', 'SM_TT', 'SM_CO', 
            'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO',
            'TRAIN_AV', 'SM_AV', 'CAR_AV']
X = df.loc[:,features]

# Initiate scaler object & fit to data
scaler = StandardScaler()  
scaler.fit(X)  

# Create new dataframe X_scaled containg the scaled features AND the (unscaled) ones: availabilities, Group, Survey, Choice
X_scaled = scaler.transform(X)

# Create the target
Y = df['CHOICE']

##### 2.2 Splitting the data in a train set and a test set<br>
Training ML models always involves a **train** and a **test** data set. The train set is used to update the weights of the model. That test set is used to evaluate the **generalisation performance** of the model.

In [ ]:
# Split the data using sk-learn's `train_test_split` function
# Note that we use 60% for training and 40% for testing
# Note that we set the random_state, in order to replicate results later (do not change) 
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, random_state = 12345, test_size = 0.4)

##### 2.3 Creating the model object<br>
A MultiLayerPerceptron (MLP) is a fully-connected feed-forward neural network. We create the MLP using `sk-learn's` MLPClassifier function. 

In [ ]:
# Create 'plain vanilla' MLP object
# Declare the number of layers and nodes per layer
# layers = (a,b) means two layer with a nodes in the 1st hidden layer and b nodes in the 2nd hidden layer
layers = (10)

# Define MLP architecture, optimiser and hyperparameters:
# We use Adam optimiser
# We use the learning rate to 0.001
# We use L2 regularisation of 0.1
# We use a batch size of 250 observations
# We use tanhg activation (transfer function)
# We set the max number of epochs to 2000
mlp = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000) 

##### 2.4 Training the model<br>
To train the MLP we use the '.fit' function.  

In [ ]:
# Train the MLP using the train data.
mlp.fit(X_train, Y_train)

In [ ]:
# It is good practice to look at the learning curve, to see how much the model has improved from the initial point,
# and how is has evolved over epochs.
fig, ax = plt.subplots(figsize = (16,8))
plt.plot(mlp.loss_curve_)
ax.set_xlabel('Epoch')
ax.set_ylabel('Cross entropy loss')
ax.grid(True,linewidth = 0.5)
ax.set_ylim(0.0,1.2)
ax.set_xlim(0,mlp.n_iter_)
ax.set_title(f'Cross entropy loss on the TRAINING DATA. \nBest CE = {mlp.loss_:4.3f}')
# The plot illustrates that the training stops when the training loss does no longer improve more that a given tolerance (e.g. 1e-6), or reaches max_iter

##### 2.5 Evaluating the model performance<br>
To evaluate the generalisation performance of our MLP, we look at the accuracy and cross-entropy on the train and test data sets.

In [ ]:
# Let's create a function that returns the accuracy and the cross entropy, for the train and test data sets
def calculate_acc_ce(mlp,X_train,Y_train,X_test, Y_test):
    
    def calculate_acc(mlp,X,Y):
        accuracy = mlp.score(X,Y)
        return accuracy

    def calculate_ce(mlp,X,Y):
        # Compute cross entropy
        # Use the model object to predict probabilities per class
        prob = mlp.predict_proba(X)

        # Multiply the probabilities with Y (0/1 array), and sum along the row axis to obtain the predicted probability of the target
        Y_dummy = pd.get_dummies(Y).to_numpy()
        prob_chosen = np.sum(prob*Y_dummy,axis=1)    
        
        # Take the logarithm
        log_prob_chosen = np.log(prob_chosen)

        # Compute the cross entropy
        cross_entropy = -np.sum(log_prob_chosen)/len(Y)
        return cross_entropy

    # Compute the accuracy
    acc_train = calculate_acc(mlp,X_train,Y_train)
    acc_test  = calculate_acc(mlp,X_test,Y_test)

    # Apply cross entropy function
    ce_train = calculate_ce(mlp,X_train,Y_train)
    ce_test = calculate_ce(mlp,X_test,Y_test)
    return acc_train, acc_test, ce_train, ce_test 

In [ ]:
# Apply our evaluation function
accuracy_train, accuracy_test, cross_entropy_train, cross_entropy_test = calculate_acc_ce(mlp,X_train,Y_train,X_test, Y_test)

# Print the results
print('\t\t Train set\t Test set')
print(f'Accuracy\t {accuracy_train:0.3f}\t\t {accuracy_test:0.3f}')
print(f'Cross entropy\t {cross_entropy_train:0.3f}\t\t {cross_entropy_test:0.3f}')

### <span style="color:skyblue">Exercise 3: Does using more nodes improve the model performance?</span> 
`A` Calculate the number of weights consumed by the current MLP with 10 hidden nodes in 1 hidden layer. <br>
`B` Retrain your model several times with {10,30,60,90} nodes. Report the cross-entropy performance on the train and test data sets.<br>
`C` Does increasing the number of nodes lead to a lower cross-entropy on the train and or test set? What is happening?<br>
`D` Suppose you would have many more choice observations from this survey. Would that enable you to develop a much better model with a cross-entropy performance of say <0.10 (on the test set)? Explain your answer. <br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

#### 3. Using Early stopping to avoid overfitting
Early stopping refers to a technique that stops the training of the network when the performance on the test data set no longer improves. Thereby, early stopping avoids the model to overfit the data. It essentially stops the training before the model can overfit.  

In [ ]:
# Note this is the same 'plain vanilla' MLP, but now we set early_stopping = True
# A validation_fraction is added. This fraction is the proportion of training data to set aside as validation set for early stopping. This data set is used to determine when to stop. 
# The training stops when the performance on the validation data set does not improve for n_iter_no_change in a row
# We use the MLP with 10 nodes and one hidden layer again.
layers = (10)
n_iter_no_change = 10
mlp_early_st = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000, early_stopping=True, n_iter_no_change = n_iter_no_change,validation_fraction = 0.25) 

# Train the MLP using the train data
mlp_early_st.fit(X_train, Y_train)

In [ ]:
# It is good practice to look at the learning curve, to see how much the model has improved from the initial point,
# and how is has evolved over epochs.
# The learning curve plot also illustrates well what early stopping does.
fig, ax = plt.subplots(figsize = (16,8))
plt.plot(mlp.loss_curve_,label=f'MLP with {mlp.hidden_layer_sizes} hidden nodes, WITHOUT early stopping')
plt.plot(mlp_early_st.loss_curve_,label=f'MLP with {mlp_early_st.hidden_layer_sizes} hidden nodes, WITH early stopping')
ax.set_xlabel('Epoch')
ax.set_ylabel('Cross entropy loss')
ax.grid(True,linewidth = 0.5)
ax.set_ylim(0.0,1.2)
ax.set_xlim(0,mlp.n_iter_)
ax.set_title('Effect of early stopping')
ax.legend()

In [ ]:
# Let's evaluate the performance of the MLP with early stopping using our evaluation function
accuracy_train_early_st, accuracy_test_early_st, cross_entropy_train_early_st, cross_entropy_test_early_st = calculate_acc_ce(mlp_early_st,X_train,Y_train,X_test, Y_test)

# Report results
print('\t\t Train set\t Test set')
print(f'Accuracy\t {accuracy_train_early_st:0.3f}\t\t {accuracy_test_early_st:0.3f}')
print(f'Cross entropy\t {cross_entropy_train_early_st:0.3f}\t\t {cross_entropy_test_early_st:0.3f}')

### <span style="color:skyblue">Exercise 4: Can we overfit with early stopping?</span> 
`A` Did early stopping reduced overfitting? How can you see this from the results?<br>
`B` Try if early stoppping also helps to avoid overfitting when using more nodes, e.g. 90 nodes

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

#### 4. Using k-fold cross validation to evaluate generalisation performance
k-fold cross validation is commonly used to more accurately **evaluate the generalisation performance** of a given network. It improves a simple train-test split approach in that it systematically cuts the data set in k pieces. k-fold cross validation is especially crucial when tuning hyperparameters (as we will see later).

In [ ]:
# Create MLP object (plain vanilla MLP without early stopping)
layers = 10
mlp_cv = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000) 

# Create scoring function
# It is necessary to create a scoring function when working with cross_validate of sk-learn
# We set `greater_is_better` to `False` as we are minimising cross entropy loss
logloss = make_scorer(log_loss, greater_is_better = False, needs_proba = True)

# Apply cross_validate, using e.g. 5 folds
# Since we use cross-validation training takes n_folds times longer than using a train-test split
n_folds = 5
cv_results = cross_validate(mlp_cv,X_scaled,Y,cv = n_folds, scoring=logloss,return_train_score=True)

In [ ]:
# Plot the train and test performance in a bar plot, for each fold
fig, ax = plt.subplots(figsize=(10, 6))
x_axis = np.arange(n_folds)
ax.bar(x_axis + -0.125, -cv_results['train_score'], color = 'b', width = 0.25,label = 'Train data set')
ax.bar(x_axis +  0.125, -cv_results['test_score'], color = 'g', width = 0.25,label = 'Test data set')
ax.set_xlabel('fold #')
ax.set_ylabel('cross entropy')
ax.grid()
ax.legend()

$\color{red}{\text{Hence, this cross validation analysis supports the finding that a plain vanilla MLP (i.e. without early stopping) overfits the data considerably}}$

#### 5. Tuning hyperparameter
When training MLPs (and most other ML models) there are several parameters we can **'tune'** (optimise) to improve the model's performance. The process of doing this is called **hyperparameter tuning**. Hyperparameter tuning can be done manually, but that is unwieldy. The `GridSearchCV` function in `sk-learn` automates the hyperparameter tuning process. When tuning the hyperparameters, it is mandatory to use a k-fold cross validation approach. Otherwise, there is a risk of overfitting on the test set *because* the parameters can be tweaked until the estimator performs optimally on the test data set.

In [ ]:
# Create MLP object (plain vanilla MLP)
mlp_gs = MLPClassifier(activation = 'tanh', solver='adam', batch_size=250, max_iter=2000)

# Define the hyperparameter search space
# 'hidden_layer_sizes' defines the number of nodes and layers
# 'alpha' governs the L2 regularisation
# 'learning_rate_init' governs the learning rate.
hyperparameter_space = {
    'hidden_layer_sizes': [(10),(10,10),(30,30)],
    'alpha': [1,1e-2,1e-4],
    'learning_rate_init': [0.01,0.001,0.0001]}

# Create scoring function
logloss = make_scorer(log_loss, greater_is_better = False, needs_proba = True)

# Create the grid_search object, with using the MLP classifier
folds = 5 # Number of cross validation splits
mlp_gridsearch = GridSearchCV(mlp_gs, hyperparameter_space, n_jobs=-1, cv=folds,scoring = logloss)

In [ ]:
# Execute the training/gridsearch
# Note that this is computationally expensive! 
# It may take up to 5 minutes, since 3 x 3 x 3 = 27 models need to be trained, each with 5 folds (=135)
mlp_gridsearch.fit(X_train, Y_train)

In [ ]:
# Save your model 
filename = 'my_tuned_model.sav'
pickle.dump(mlp_gridsearch, open(data_folder/filename,'wb'))

In [ ]:
# Load your model (if you have a saved model)
# mlp_gridsearch = pickle.load(open(data_folder/filename,'rb'))

Visualise the hyperparameter tuning results

In [ ]:
# Collect results into a new pandas dataframe
df_gridsearch = pd.DataFrame.from_dict(mlp_gridsearch.cv_results_)
    
# Add new column with a label for the hyperparameter combinations %% NOT SURE IF THIS IS BET WAY TO DO THIS IN PYTHON   
df_gridsearch['gs_combinations'] = 'L2 = '+ df_gridsearch['param_alpha'].astype('str') + '; Learning_rate = '+ df_gridsearch['param_learning_rate_init'].astype('str') + '; Layers = ' + df_gridsearch['param_hidden_layer_sizes'].astype('str')
df_gridsearch = df_gridsearch.sort_values('rank_test_score')

# Visualise deviation in performance across hyper parameter settings
plt.figure(figsize = (16,6))
ax = sns.barplot(x = df_gridsearch.gs_combinations,y=-df_gridsearch.mean_test_score,palette="Blues_d",)
plt.xticks(rotation=90)

print('Best hyperparameters found:\t', mlp_gridsearch.best_params_)
print('Best model performance:\t\t', -mlp_gridsearch.best_score_)

### <span style="color:skyblue">Exercise 5:  Hyperparameter tuning</span> 
`A` What hyperparameter turns out to be particularly impactful on the model performance?<br>
`B` Can you think of reasons why this could be the case?


In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

**(Re)Training the model with optimised hyperparameters**<br> 
After completing hypertuning, you know the optimal hyperparameters. <br>
Therefore, after hypertuning we always retrain the model using the optimised hyperparameters.

In [ ]:
# Create a new mlp object using the optimised hyperparameters, just using the train/test split
layers = mlp_gridsearch.best_params_['hidden_layer_sizes']
lr = mlp_gridsearch.best_params_['learning_rate_init']
alpha = mlp_gridsearch.best_params_['alpha']
mlp_gs = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = lr, alpha=alpha, batch_size=250, activation = 'tanh', max_iter = 2000) 

# Train the model
mlp_gs.fit(X_train,Y_train)

In [ ]:
# Let's also evaluate performance of the hypertuned model using our evaluation function
accuracy_train_gs, accuracy_test_gs, cross_entropy_train_gs, cross_entropy_test_gs = calculate_acc_ce(mlp_gs,X_train,Y_train,X_test, Y_test)

# Report results
print('\t\t Train set\t Test set')
print(f'Accuracy\t {accuracy_train_gs:0.3f}\t\t {accuracy_test_gs:0.3f}')
print(f'Cross entropy\t {cross_entropy_train_gs:0.3f}\t\t {cross_entropy_test_gs:0.3f}')

#### 6. Evaluating and comparing performances across trained models

To evaluate the performance of a wide range is metrics are available, beyond generalisation performance.<br>
Here, we look at:<br>
i. Confusion matrix<br>
ii. Precision and Recall<br>
iii. Matthew's correlation coefficient

**i) Confusion matrix**<br>
Confusion matrices shows counts from predicted and actual outcomes. The counts on the diagonal are correctly classified outcomes (the model predictions and the ground truth are the same). The counts on the off-diagonal elements are the misclassified outcomes. Hence, the best classifier will have a confusion matrix with only diagonal elements and the rest of the elements set to zero.

In [ ]:
# Predict the choices for the test data set, using the MLP trained with early stopping and MLP with hyperparameters tuned
Y_pred_early_st  = mlp_early_st.predict(X_test)  # 0/1 predictions of MLP trained with early stopping
Y_pred_gs = mlp_gs.predict(X_test)                      # 0/1 predictions of MLP with hyperparameters tuned

# Show the confusion matrices, to compare the hyperparameter tuned network with the early stopping network
fig, ax = plt.subplots(2,2,figsize = (16,12))
fig.set_tight_layout(True)
ylabels = ['Train', 'SM', 'Car']
cm1 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_early_st, display_labels = ylabels, normalize=None,  ax=ax[(0,0)])
cm2 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_early_st, display_labels = ylabels, normalize='true',ax=ax[(1,0)])
cm3 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_gs, display_labels = ylabels, normalize=None,  ax=ax[(0,1)])
cm4 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_gs, display_labels = ylabels, normalize='true',ax=ax[(1,1)])

# Add titles
cm1.ax_.set_title(f'MLP with {mlp_early_st.hidden_layer_sizes} nodes \n trained with early stopping')
cm2.ax_.set_title(f'MLP with {mlp_early_st.hidden_layer_sizes} nodes \n trained with early stopping')
cm3.ax_.set_title(f'MLP with {mlp_gs.hidden_layer_sizes} nodes \n hyperparameters tuned')
cm4.ax_.set_title(f'MLP with {mlp_gs.hidden_layer_sizes} nodes \n hyperparameters tuned')

### <span style="color:skyblue">Exercise 6:  Model accuracy</span> 
Accuracy is defined as the true positives over the total number of cases.<br>
`A` Manually compute the prediction accuracy of the model with early stopping and the model with hyperparameter tuning<br>
`B` For which class (Train/SM/Car) does the hyperparameter tuning improves the prediction accuracy most?<br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

**ii) Precision and Recall**<br>
Looking at the confusion matrices, the improvements in prediction accuracy due to the hyperparameter tuning may not seem very impressive. However, one should keep in mind that 0/1 predictions are sensitive to class imbalances, which are present in these data. Moreover, accuracy can be a misleading metric for imbalanced data sets. A naive model that would simply always predict "SM" will already do quite good.

To assess the model performance in more depth, we thus must look at the predictions at the level of the classes.<br> 
Next, we compute Precision and Recall.<br>
* **Precision** Tells you what fraction of predictions for a given class are actually of that class.<br>
* **Recall** Tells what fraction of all observations belonging to a given class are correctly predicted as such by the model. Recall is also known as True Positive Rate (TPR), Sensitivity, Probability of Detection. <br>

In [ ]:
# To calculate the precision and recal score we conveniently use sk-learn's 'classification_report' functionality
print('Classification report for plain vanilla MLP with early stopping\n',classification_report(Y_test,Y_pred_early_st, target_names= ylabels))
print('\nClassification report for plain vanilla MLP with hyperparameters tuned\n',classification_report(Y_test,Y_pred_gs, target_names= ylabels))

**iii) Matthew's correlation coefficient**<br>
Another commonly used metric to evaluate the prediction performance while accounting for imbalances in the data set is  Matthew's correlation coefficient. Matthew's Correlation Coefficient (MCC) is generally regarded as being one of the best measures to describe the confusion matrix of true and false positives and negatives by a **single number**, even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction.

In [ ]:
# Taking into account the imbalances of the data
print(f'Matthews correlation coefficient for plain Vanilla MLP with early stopping:\t {matthews_corrcoef(Y_test, Y_pred_early_st):4.3f}')
print(f'Matthews correlation coefficient for plain Vanilla MLP with hypertuning:\t {matthews_corrcoef(Y_test, Y_pred_gs):4.3f}')

### <span style="color:skyblue">Exercise 7:  Model precision, recall, and Matthew's correlation coefficent</span> 
`A` Compare and interpret the results from the classifications reports between the early stopping and hypertuned model.<br>
`B` Compare and interpret the results from Matthew's correlation coefficient between the early stopping and hypertuned model.

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)